In [1]:
# Import cependencies
import os   
import pandas as pd
import gmaps
import requests
# Import the API Key
from config import gKey

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
fileToRead = os.path.join("weather_data", "cities.csv")
cityDataDF = pd.read_csv(fileToRead)
cityDataDF.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2020-07-11 15:10:20,-54.80,-68.30,32.00,91,75,3.36
1,1,Punta Arenas,CL,2020-07-11 15:10:20,-53.15,-70.92,35.01,86,75,11.41
2,2,Mataura,NZ,2020-07-11 15:10:21,-46.19,168.86,44.08,91,100,3.51
3,3,Butaritari,KI,2020-07-11 15:10:21,3.07,172.79,82.42,80,100,15.55
4,4,Mahebourg,MU,2020-07-11 15:10:21,-20.41,57.70,71.60,68,75,12.75


In [3]:
# Check data types 
cityDataDF.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gKey)

In [5]:
# Get the maximum temperature. (Avoid error in the heatmap_layer function as weight cannot be <0)
maxTemp = cityDataDF["Max Temp"]
temps = []
for temp in maxTemp:
    temps.append(max(temp,0))

In [6]:
# Heat map for Max Tempertures
# ---------------------------------
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = cityDataDF[["Lat","Lng"]]
# 2. Assign the weights variable to some values.
# Get the maximum temperature. (Avoid error in the heatmap_layer function as weight cannot be <0)
temperatures = [max(temp,0) for temp in cityDataDF["Max Temp"]]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heatLayer = gmaps.heatmap_layer(locations, weights=temperatures, dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heatLayer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heat map for Percent Humidity
# ---------------------------------
locations = cityDataDF[["Lat","Lng"]]
humidity = cityDataDF["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heatLayer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heat map for Percent Cloudiness
# ---------------------------------
locations = cityDataDF[["Lat","Lng"]]
clouds = cityDataDF["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heatLayer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heat map for Wind Speed
# ---------------------------------
locations = cityDataDF[["Lat","Lng"]]
wind = cityDataDF["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heatLayer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
minTempInput = float(input("What is the minimum temperature you would like for your trip? "))
maxTempInput = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferredCitiesDF = cityDataDF.loc[(cityDataDF["Max Temp"]>=minTempInput) & (cityDataDF["Max Temp"]<=maxTempInput)]
preferredCitiesDF.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Butaritari,KI,2020-07-11 15:10:21,3.07,172.79,82.42,80,100,15.55
13,13,Kutum,SD,2020-07-11 15:10:22,14.20,24.67,83.23,41,53,6.22
15,15,Touros,BR,2020-07-11 15:10:19,-5.20,-35.46,78.80,74,75,13.87
17,17,Albany,US,2020-07-11 15:06:44,42.60,-73.97,84.00,74,75,1.01
20,20,Kapaa,US,2020-07-11 15:10:09,22.08,-159.32,78.80,83,90,12.75


In [12]:
# Check if the filtered data contains null values
preferredCitiesDF.count()

City_ID       178
City          178
Country       177
Date          178
Lat           178
Lng           178
Max Temp      178
Humidity      178
Cloudiness    178
Wind Speed    178
dtype: int64

In [13]:
# Eliminate null values
preferredCitiesDF = preferredCitiesDF.dropna()
preferredCitiesDF.count()

City_ID       177
City          177
Country       177
Date          177
Lat           177
Lng           177
Max Temp      177
Humidity      177
Cloudiness    177
Wind Speed    177
dtype: int64

In [14]:
# Create DataFrame called hotelDF to store hotel names along with city, country, max temp, and coordinates.
hotelDF = preferredCitiesDF[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotelDF["Hotel Name"] = ""
hotelDF.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,82.42,3.07,172.79,
13,Kutum,SD,83.23,14.20,24.67,
15,Touros,BR,78.80,-5.20,-35.46,
17,Albany,US,84.00,42.60,-73.97,
20,Kapaa,US,78.80,22.08,-159.32,


In [15]:
# Set base url
baseUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Set parameters to search for a hotel.
params = {"key": gKey, "radius": 5000, "type": "lodging"}

# Iterate through the DataFrame.
for index, row in hotelDF.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(baseUrl, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        # Grab the first hotel from the results and store the name.
        hotelDF.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in city '{row['City']}'... skipping.")


Hotel not found in city 'Albany'... skipping.
Hotel not found in city 'Bondo'... skipping.
Hotel not found in city 'Birao'... skipping.
Hotel not found in city 'Caluquembe'... skipping.
Hotel not found in city 'Luzon'... skipping.
Hotel not found in city 'Zilair'... skipping.
Hotel not found in city 'Ler'... skipping.
Hotel not found in city 'Lebyazhye'... skipping.
Hotel not found in city 'Chapayevka'... skipping.
Hotel not found in city 'Taguatinga'... skipping.
Hotel not found in city 'Garoafa'... skipping.
Hotel not found in city 'Ndende'... skipping.
Hotel not found in city 'Bonfim'... skipping.
Hotel not found in city 'Barra Patuca'... skipping.
Hotel not found in city 'Lusambo'... skipping.


In [16]:
hotelDF.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,82.42,3.07,172.79,Isles Sunset Lodge
13,Kutum,SD,83.23,14.20,24.67,Dibbo areaمنطقة دبو
15,Touros,BR,78.80,-5.20,-35.46,INN NEW HORIZON
17,Albany,US,84.00,42.60,-73.97,
20,Kapaa,US,78.80,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [18]:
# Heatmap from hotel DataFrame
# ------------------------------------

locations = hotelDF[["Lat","Lng"]]
# Get the maximum temperature. (Avoid error in the heatmap_layer function as weight cannot be <0)
temperatures = [max(temp,0) for temp in hotelDF["Max Temp"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatLayer = gmaps.heatmap_layer(locations, weights=temperatures, dissipating=False, max_intensity=300, point_radius=4)


infoBoxTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotelInfo = [infoBoxTemplate.format(**row) for index, row in hotelDF.iterrows()]

markerLayer = gmaps.marker_layer(locations,info_box_content=hotelInfo)
fig.add_layer(markerLayer)
fig.add_layer(heatLayer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))